In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from ydata_profiling import ProfileReport
import warnings

In [2]:
df_cash = pd.read_csv("./project_dataset/extract - cash request - data analyst.csv")
df_fees = pd.read_csv("./project_dataset/extract - fees - data analyst - .csv")

In [3]:
warnings.filterwarnings("ignore")

# 1. Frecuencia de Uso del Servicio:
Definición: Queremos entender cuántas veces los usuarios dentro de cada cohorte (posiblemente por meses o años) usan el servicio de adelanto de efectivo.
Cálculo:
Agrupa los registros de df_cash por el campo user_id y el periodo (por ejemplo, mes o trimestre) basado en created_at.
Cuenta el número de transacciones por usuario dentro de cada cohorte.
Calcula la media de las transacciones por usuario para tener una idea de la frecuencia de uso promedio dentro de cada cohorte.

In [5]:
# Convertir la columna `created_at` a formato datetime
df_cash['created_at'] = pd.to_datetime(df_cash['created_at'])

# Crear una nueva columna 'cohorte' basado en el mes de creación
df_cash['cohorte'] = df_cash['created_at'].dt.to_period('M')

# Agrupar por cohorte y usuario para calcular la frecuencia de uso
frecuencia_uso = df_cash.groupby(['cohorte', 'user_id']).size().reset_index(name='uso_total')

# Calcular la media de frecuencia de uso por cohorte
frecuencia_cohorte = frecuencia_uso.groupby('cohorte')['uso_total'].mean().reset_index(name='frecuencia_media_uso')

In [6]:
print(frecuencia_uso)
print(frecuencia_cohorte)

       cohorte   user_id  uso_total
0      2019-11      47.0          1
1      2019-12      35.0          2
2      2019-12      47.0          1
3      2019-12      52.0          1
4      2019-12      53.0          1
...        ...       ...        ...
18795  2020-11  103425.0          1
18796  2020-11  103437.0          1
18797  2020-11  103496.0          1
18798  2020-11  103503.0          1
18799  2020-11  103719.0          1

[18800 rows x 3 columns]
    cohorte  frecuencia_media_uso
0   2019-11              1.000000
1   2019-12              1.121951
2   2020-01              1.106918
3   2020-02              1.068027
4   2020-03              1.131148
5   2020-04              1.132791
6   2020-05              1.143082
7   2020-06              1.226703
8   2020-07              1.306452
9   2020-08              1.296141
10  2020-09              1.144147
11  2020-10              1.072683
12  2020-11              1.000000


# 2. Tasa de Incidentes:
Definición: Queremos identificar incidentes en el dataset df_fees, como pagos rechazados o direct debits fallidos, y calcular la tasa de incidentes por cohorte.
Cálculo:
Filtra los registros en df_fees que correspondan a incidentes (puedes usar la columna category para filtrar aquellos que tienen categorías relacionadas con incidentes).
Agrupa los incidentes por cohorte (nuevamente, usando la fecha de created_at en df_cash para los IDs relacionados).
Calcula la tasa de incidentes dividiendo el número de incidentes entre el número total de transacciones de cada cohorte.

In [8]:
df_fees['category'].unique()

array([nan, 'rejected_direct_debit', 'month_delay_on_payment'],
      dtype=object)

In [9]:
# Filtrar registros de incidentes en df_fees
incidentes = df_fees[df_fees['category'].isin(['rejected_direct_debit', 'month_delay_on_payment'])]

# Unir df_fees con df_cash para obtener el campo de cohorte
incidentes = incidentes.merge(df_cash[['id', 'cohorte']], left_on='cash_request_id', right_on='id')

# Calcular el número de incidentes por cohorte
incidentes_cohorte = incidentes.groupby('cohorte').size().reset_index(name='total_incidentes')

# Calcular el número total de transacciones por cohorte
total_transacciones = df_cash.groupby('cohorte').size().reset_index(name='total_transacciones')

# Calcular la tasa de incidentes
tasa_incidentes = incidentes_cohorte.merge(total_transacciones, on='cohorte')
tasa_incidentes['tasa_incidentes_%'] = tasa_incidentes['total_incidentes'] / tasa_incidentes['total_transacciones'] * 100

In [10]:
tasa_incidentes

,cohorte,total_incidentes,total_transacciones,tasa_incidentes_%
0,2020-05,44,837,5.256870
1,2020-06,560,2615,21.414914
2,2020-07,659,3601,18.300472
3,2020-08,498,3417,14.574188
4,2020-09,385,4221,9.121061
5,2020-10,50,7725,0.647249


# 3. Ingresos Generados por la Cohorte:
Definición: Queremos calcular el total de ingresos generados por cada cohorte.
Cálculo:
Agrupa df_fees por cohorte (relacionando los IDs de df_cash con df_fees mediante cash_request_id).
Suma el total_amount para cada cohorte

In [12]:
# Unir df_fees con df_cash para obtener el campo de cohorte
ingresos = df_fees.merge(df_cash[['id', 'cohorte']], left_on='cash_request_id', right_on='id')

# Calcular los ingresos por cohorte sumando el total_amount
ingresos_cohorte = ingresos.groupby('cohorte')['total_amount'].sum().reset_index(name='ingresos_totales')


In [13]:
ingresos_cohorte

,cohorte,ingresos_totales
0,2020-04,5.0
1,2020-05,1285.0
2,2020-06,8725.0
3,2020-07,10395.0
4,2020-08,17565.0
5,2020-09,22935.0
6,2020-10,43815.0
7,2020-11,565.0


# 4. Nueva Métrica Relevante - Tasa de Reembolsos por Cohorte:
Definición: Una métrica relevante para el análisis podría ser la tasa de reembolsos exitosos (pagos que han sido reembolsados en la categoría money_back).
Cálculo:
Filtra las transacciones con estado money_back.
Agrupa por cohorte y cuenta cuántos reembolsos exitosos se han hecho.
Divide el número de reembolsos por el total de transacciones de cada cohorte.

In [15]:
# Filtrar transacciones con estado `money_back`
reembolsos = df_cash[df_cash['status'] == 'money_back']

# Calcular el número de reembolsos por cohorte
reembolsos_cohorte = reembolsos.groupby('cohorte').size().reset_index(name='total_reembolsos')

# Calcular la tasa de reembolsos por cohorte
tasa_reembolsos = reembolsos_cohorte.merge(total_transacciones, on='cohorte')
tasa_reembolsos['tasa_reembolsos_%'] = tasa_reembolsos['total_reembolsos'] / tasa_reembolsos['total_transacciones'] * 100


In [16]:
tasa_reembolsos

,cohorte,total_reembolsos,total_transacciones,tasa_reembolsos_%
0,2019-12,154,289,53.287197
1,2020-01,102,223,45.739910
2,2020-02,102,184,55.434783
3,2020-03,165,244,67.622951
4,2020-04,309,473,65.327696
5,2020-05,591,837,70.609319
6,2020-06,1591,2615,60.841300
7,2020-07,1909,3601,53.013052
8,2020-08,1922,3417,56.248171
9,2020-09,2618,4221,62.023217
